## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
#Загрузим датасет и посмотрим информацию
import pandas as pd
from IPython.display import display
data = pd.read_csv('/datasets/data.csv')
display(data)
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Получив данные, нас в первую очередь могут привлечь отрицательные значения в столбце 'days_employed', от которых в дальнейшем необходимо будет избавиться. Далее необходимо проверить, есть ли пропуски в нашей таблице или какие-то другие значения, которые могут исказить наши данные.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#сгруппируем наш датасет по типу занятости и доходу и найдем среднее значение
data_income_mean = data.groupby('income_type')['total_income'].mean()
#сгруппируем наш датасет по типу занятости и рабочим и найдем среднее значение
days_employed_mean = data.groupby('income_type')['days_employed'].mean()
#проверяем
#print(data_income_mean)
#print(days_employed_mean)
#заменим пропущенные значения на получившиеся средние для каждого вида занятости
data['total_income'] = data['total_income'].fillna(('data_income_mean'))
data['days_employed'] = data['days_employed'].fillna(('days_employed_mean'))
#проверим остались ли пропущенные значения
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Обнаружились пропущенные значения по столбцам days_employed и total_income. Это можно объяснить тем, что если у клиента нет дохода, соответсвенно значения дней работы также будут пустыми. Пропущенные значения были заменены на 0, чтобы ничего не отразилось на качестве данных. Также проверили столбцы children и family_status на отсутсвие ошибочных значений.

### Замена типа данных

In [3]:
# Шаг 1. Заполним все пропуски значениями Nan:
data['days_employed'] = pd.to_numeric(data['days_employed'], errors='coerce')
data['total_income'] = pd.to_numeric(data['total_income'], errors='coerce')


### Вывод

Так как данные были нам представлены в смешанном виде,для их обработки необходимо было привести их в тип float64. 

### Обработка дубликатов

In [4]:
# Произведём ручной поиск дубликатов. Выберем столбцы, где чаще всего есть разные значения:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()

# Далее найдём дубликаты в каждом столбце датафрейма
print(data['children'].value_counts()) 
print(data['days_employed'].value_counts()) 
print(data['dob_years'].value_counts()) 
print(data['education'].value_counts()) 
print(data['education_id'].value_counts()) 
print(data['family_status'].value_counts())  
print(data['family_status_id'].value_counts()) 
print(data['gender'].value_counts()) 
print(data['income_type'].value_counts()) 
print(data['debt'].value_counts()) 
print(data['total_income'].value_counts()) 
print(data['purpose'].value_counts())  
# Удалим все дубликатов из таблицы data специальным методом
data = data.drop_duplicates().reset_index(drop=True)


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64
среднее   

### Вывод

 Поиск дублей показал дублированные записи по столбцу purpose. Необходимо будет произвести лемматизацию этого стобца.

### Лемматизация

In [5]:
# Лемматизировать будем столбец purpose, который дал нам большое количество дубликатов и создадим для него отдельный столбец
#загружем библиотеку для лемматизации
from pymystem3 import Mystem 
m = Mystem() 
def do_lemma(row):
    lemma = m.lemmatize(row)
    return lemma
#создадим дополнительный столбец с лемматизированными целями
data['purpose_lemma'] = data['purpose'].apply(do_lemma)
#создадим строку из уникальных целей
print(data['purpose_lemma'])


0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21449                  [операция,  , с,  , жилье, \n]
21450               [сделка,  , с,  , автомобиль, \n]
21451                              [недвижимость, \n]
21452    [на,  , покупка,  , свой,  , автомобиль, \n]
21453             [на,  , покупка,  , автомобиль, \n]
Name: purpose_lemma, Length: 21454, dtype: object


### Вывод

В ходе лемматизации получили популярные запросы на получение кредита: недвижимость, жилье, автомобиль, образование, свадьба. 

### Категоризация данных

In [6]:
# Категоризацию данных построим по зависимости количества детей от возможной задолженности
def children_and_debt(row): #создадим функцию, которая будет категоризировать должник клиент или нет по количеству детей
    children = row['children']
    if 1 <= children :
            return 'с детьми'
    if 1 > children :
            return 'без детей'
# Протестируем   
row_values = [0]
row_columns = ['children'] 
row = pd.Series(data=row_values, index=row_columns)
print(children_and_debt(row))
#создаем новый столбец, чтобы проверить работу функции
data['children_return'] = data.apply(children_and_debt, axis=1)
#создадим сводную таблицу
data_children_pivot = data.pivot_table(index = 'children_return', columns = 'debt',values = 'children', aggfunc = 'count')
print(data_children_pivot)



без детей
debt                 0     1
children_return             
без детей        13074  1064
с детьми          6639   677


### Вывод

Результат нам показал, что клиенты, у которых нет детей чаще погашают кредит в срок, значит первый вывод можно сделать, что может наблюдаться зависимость, если в семье нет детей, то клиент надежный плательщик.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
#создадим переменные, которые показывают наличие детей у клиентов
children_yes = data[data['children'] != 0]
children_no = data[data['children'] == 0]
#найдем отношение наличия задолженности к наличию детей
children_debt = children_yes['debt'].sum() / len(children_yes)
children_no_debt = children_no['debt'].sum() / len(children_no)
print('Процент клиентов с детьми, у которых задолжность: {:.1%}'.format(children_debt))
print('Процент бездетных, у которых задолжность: {:.1%}'.format(children_no_debt))

Процент клиентов с детьми, у которых задолжность: 9.2%
Процент бездетных, у которых задолжность: 7.5%


### Вывод

Данные показывают, что бездетные клиенты чаще выплачивают кредит вовремя, чем те, у которых есть дети. Хотя казалось, что наличие детей делает людей более ответсвенными. Это подтвердило нашу первую гипотезу из категоризации данных

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
# сопоставим уникальные значения с семейным статусом и их ID
print(data['family_status'].unique())
print(data['family_status_id'].unique())
#создаем таблицу data_pivot_family_status где смотрим должников по категории семейного положения
data_pivot_family_status = data.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count')
#считаем процент должников
#print(data_pivot_family_status)
data_pivot_family_status['sum'] = data_pivot_family_status[1] + data_pivot_family_status[0]
data_pivot_family_status['coef %'] = (data_pivot_family_status[1] / data_pivot_family_status['sum']) * 100
#cчитаем сумму клиентов каждой категории
print(data_pivot_family_status[['coef %', 'sum']])

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']
[0 1 2 3 4]
debt                     coef %    sum
family_status                         
в разводе              7.112971   1195
вдовец / вдова         6.569343    959
гражданский брак       9.347145   4151
женат / замужем        7.545182  12339
не женат / не замужем  9.750890   2810


### Вывод

Как видно из результатов семейное положение влияет при возрвате кредита. Люди, которые не состояли в официальном браке, т.е. не женатые или находящиеся в гражданском браке чаще остаются в должниках

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
# Сопоставим доход и их наличие задолженности. 
def total_income_and_debt(row):
#Доход до 30:   
    if 1 < row <= 30000:
            return 'до 30'
#Доход от 30 до 50:        
    if 30000  < row <= 50000:
            return 'от 30 до 50'     
#Доход от 50 до 100:       
    if 50000  < row <= 100000:
            return 'от 50 до 100'     
#Доход от 100 до 150:       
    if 100000  < row <= 150000:
            return 'от 100 до 150'     
#Доход от 150 до 200:        
    if 150000  < row <= 200000:
            return 'от 150 до 200'
#Доход от 200 и более:        
    if 200000  < row:
            return 'от 200'
# Протестируем  
data['category_income'] = data['total_income'].apply(total_income_and_debt)
data_pivot_income = data.pivot_table(index='category_income', columns='debt', values='total_income', aggfunc='count')
#считаем процент должников
data_pivot_income['sum'] = data_pivot_income[1] + data_pivot_income[0]
data_pivot_income['coef %'] = (data_pivot_income[1] / data_pivot_income['sum']) * 100
#cчитаем сумму клиентов каждой категории
print(data_pivot_income[['coef %', 'sum']])


debt               coef %   sum
category_income                
до 30            9.090909    22
от 100 до 150    8.607994  5704
от 150 до 200    8.936377  4118
от 200           7.066719  5066
от 30 до 50      6.000000   350
от 50 до 100     8.090931  4091


### Вывод

Уровень дохода также влияет на возврат кредита в срок. Так, люди с низким достатком чаще не погашают свою задолженность вовремя. Также чаще остаются в должниках клиенты с относительно высоким достатком от 100 до 200 т.р.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
#Применим результаты лематизации
#обозначим жилье - 1
#автомобиль - 2
#образование - 3
#жилье - 4
#недвижимость - 5
def purpose_lemma(row):
    if 'свадьба' in row:
        return 1
    elif 'автомобиль' in row:
        return 2
    elif 'образование' in row:
        return 3
    elif 'жилье' in row:
        return 4
    elif 'недвижимость' in row:
        return 5
    else:
        return 6
#протестируем  
data['purpose_lemma'] = data['purpose_lemma'].apply(purpose_lemma)
purpose_table = data.groupby(['purpose_lemma'], as_index = False).agg({'debt':'sum', 'purpose': 'count'})
#создаем новый столбец, чтобы проверить работу функции
data_pivot_purpose = data.pivot_table(index='purpose_lemma', columns='debt', values='purpose', aggfunc='count')
#print(data_pivot_purpose) 
#считаем процент должников
data_pivot_purpose['sum'] = data_pivot_purpose[1] + data_pivot_purpose[0]
data_pivot_purpose['coef %'] = (data_pivot_purpose[1] / data_pivot_purpose['sum'])*100
print(data_pivot_purpose[['coef %','sum']])


debt             coef %   sum
purpose_lemma                
1              8.003442  2324
2              9.359034  4306
3              9.220035  4013
4              6.905830  4460
5              7.463392  6351


### Вывод

Одной из самых "безобидных" целей, на которую кредит возвращается вовремя является приобретение жилья. А наборот категории, по которым процент должников наибольший приобретение автомобиля и образование

### Шаг 4. Общий вывод

1. Наблюдается зависимость от количества детей и возврата кредита в срок: клиенты с 1 и ребенком реже возвращают долг, чем бездетные клиенты.
2. Видна зависимость между семейным статусом и возвратом кредита в срок: люди, находящиеся в браке более ответственно подходят к этому делу и реже становятся должниками, чем неженатые или находящиеся в разводе.
3. Зависимоть по уровень дохода показала, что клиенты с доходом доходом ниже 30 т.р. а также с доходом от 100 до 200 т.р. чаще остаются должниками
4. Клиенты, взявшие кредит на крупные покупки, такие как жилье или другая недвижимость реже оказываются в должниках, чем клиенты, взявшие кредит на более "дешевые" цели такие как приобретение автомобиля или оплата образования.
 Таким образом мы выяснили, что все 4 фактора оказывают зависимость к возврату долга в банке. 

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.